## GLACIER AREA CALCULATOR ### 

This project will primarily use the USGS Python Package to identify, download and present Landsat imagery from 1985, 2005 and 2025 across several glacier mountain ranges across the world. 
The mountain ranges will include:
- Kenai Mountains (Alaska)
- Swiss Alps
- *Insert Greenland/Himalayas maybe?*
This script will download and present the images in False Colour Composite, then will generate a classification to identify the ice area on the images and this classification will be used in each image to calculate the change in glacier area across glaciated areas on a global scale.

##TO DO LIST FOR THE STUDY## 


----KENAI MOUNTAINS----
- Identify Landsat information on the USGS EE site and get the correct WRS path, etc.
- Download Landsat Images for Kenai Mountains first
- Clasification
- Area change calculation
- See if you can edit the image to show only the selected mountain range (Crop Stacked Landsat TIF)
- Maybe include some help() but ask Bob if it is something that is expected or required

----Swiss Alps-----
- Same as above
- Try to select area with Alestchgletscher

----GREENLAND/HIMALAYAS----
- Same as above


---AFTER LANDSAT DONE-----
- Generate a comparison Table/Template for all results acquired
- IF YOU HAVE TIME, maybe do a couple more glacier mountains
- Keep it that the script can be used to generate area as long as the correct info is input (ID, WRS, ETC..)
- Ask if an ArcGIS element is expected for the assignment
- Ask/Check how the how-to-guide is to be formatted (separate word doc in Git Repos/in the README/....
- Check how the final repos is to be submitted



In [1]:
import usgs

In [2]:
from usgs import api

In [3]:
# Open and manually read the username from the .usgs_user file in the home directory
with open('C:/Users/couse/.usgs_user', 'r') as usertext:
    username = usertext.read()

#Open and manually read the password/token from the.usgs_token in the home directory
with open('C:/Users/couse/.usgs_token', 'r') as tokentext:
    password = tokentext.read()
    
# Login to the API using the username and the token(API_KEY provided by the USGS) as password
login = api.login(username, password, save=True)

#Below the API Key (token) is extracted separately as it will be used for the functions later in the script
api_key = login['data']

In [4]:
api.logout()

{'requestId': 1982787590,
 'version': 'stable',
 'sessionId': 314954733,
 'data': None,
 'errorCode': None,
 'errorMessage': None}